# Libraries

In [ ]:
!pip install Sastrawi
!pip install clean-text
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=c00e8b9bc1e172385c62533c3dbc45d60d28b74c3c662d04ee5298183a638685
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from cleantext import clean
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import sklearn

In [ ]:
# !pip freeze > requirements.txt

# Get Data from ES and prepare DF

In [ ]:
import requests
import json

In [ ]:
elasticsearch_url = 'https://es-1.datanusantara.com:9201'
username = 'adminkibana'
password = 'kibanaPASS!@#2019'
index_endpoint = 'news-crawl-production-v2'

In [ ]:
query = {
  "_source": ["target", "title","description_content"],
  "size": 600,
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "bool": {
            "should": [


              {
                "match_phrase": {
                  "target": "banggai"
                }
              }
            ],
            "minimum_should_match": 1


          }
        }
      ]
    }
  }
}
search_url = f"{elasticsearch_url}/{index_endpoint}/_search"
response = requests.get(search_url, json=query, auth=(username, password))

search_url

'https://es-1.datanusantara.com:9201/news-crawl-production-v2/_search'

In [ ]:
search_results = response.json()
# print(json.dumps(search_results, indent=2))
print(len(search_results['hits']['hits']))

600


In [ ]:
# Parse the response to extract relevant data
search_results = response.json()

# Process the data and create a DataFrame
data = []
for hit in search_results.get("hits", {}).get("hits", []):
    target = hit["_source"]["target"]
    title = hit["_source"]["title"]
    text = hit["_source"]["description_content"]

    data.append({"target": target,  "title": title, "text": text})

# Create a DataFrame
df = pd.DataFrame(data)
print(df.head())

    target                                              title  \
0  banggai  Mobil Sim Keliling Polres Banggai Meriahkan Pa...   
1  banggai  Film Horor “Piara Mayat”,  Kreativitas Anak Mu...   
2  banggai  Festival Malabot Tumbe 2023 Kembali Meriahkan ...   
3  banggai  Rakor Kampanye Damai Pemilu 2024, Polres Bangg...   
4  banggai  Bawaslu Banggai Perkuat Koordinasi dengan Panw...   

                                                text  
0  KABAR LUWUK –  Suasana Pasar Murah bersubsidi ...  
1  KABAR LUWUK  –  Film Horor“Piara Mayat”,Kreati...  
2  KABAR LUWUK  –Festival Malabot Tumbe 2023Kemba...  
3  “Kerja Cepat dan Terkoordinir dalam Penerbitan...  
4  KABAR LUWUK  –Bawaslu BanggaiPerkuat Koordinas...  


In [ ]:
# Drop Unnamed column
df.drop(df.columns[df.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

In [ ]:
df.head()

,target,title,text
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati..."
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...


# Preprocessing

## Delete Duplicates

In [ ]:
df.duplicated(subset='title').sum()


14

In [ ]:
df = df.drop_duplicates(subset='title')

## Deleting Blank (Nan) Rows

In [ ]:
df = df[df['text'] != '']

In [ ]:
len(df)

579

In [ ]:
#row_index = 127  # Index of the row you want to print (Python indexing starts from 0)
#columns_to_print = ['title', 'text']  # List of column names to display

# Use .loc[] to select the row by its index and specify the columns to display
#row_data = df.loc[row_index, columns_to_print]

#print(row_data)

## Case Folding

In [ ]:
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah dicasefolding
df['lwr'] = df['text'].apply(clean_lower)
df['lwr']

In [ ]:
df.head()

In [ ]:
print(df['text'][30])
print("_______________________________________________________________________________________________________________")
print(df['lwr'][30])

## Cleansing

### Remove Numbers

In [ ]:
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

In [ ]:
df['clean_number'] = df['lwr'].apply(remove_numbers)
df['clean_number']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe kembali m...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
594    banggai raya-satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu.com, asnawi zikri ...
596    banggai raya-lebih  pebulutangkis dari kabupat...
598    laporan wartawan tribunpalu.com, asnawi zikri ...
599    laporan wartawan tribunpalu.com, asnawi zikri ...
Name: clean_number, Length: 579, dtype: object

In [ ]:
df.tail()

,target,title,text,lwr,clean_number
594,banggai,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...
595,banggai,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
596,banggai,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...
598,banggai,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
599,banggai,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."


### Remove Punctuation

In [ ]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

In [ ]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
df['clean_punct'] = df['clean_number'].apply(clean_punct)
df['clean_punct']

0      kabar luwuk    suasana pasar murah bersubsidi ...
1      kabar luwuk     film horor piara mayat kreativ...
2      kabar luwuk   festival malabot tumbe kembali m...
3       kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk   bawaslu banggaiperkuat koordinas...
                             ...                        
594    banggai raya satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu com asnawi zikri t...
596    banggai raya lebih  pebulutangkis dari kabupat...
598    laporan wartawan tribunpalu com asnawi zikri t...
599    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_punct, Length: 579, dtype: object

In [ ]:
df.head()


,target,title,text,lwr,clean_number,clean_punct
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...


### Delete whitespaces

In [ ]:
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
df['clean_double_ws'] = df['clean_punct'].apply(normalize_whitespace)
df['clean_double_ws']

0      kabar luwuk suasana pasar murah bersubsidi di ...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe kembali mer...
3      kerja cepat dan terkoordinir dalam penerbitan ...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
594    banggai raya satu gudang bahan aluminium di ja...
595    laporan wartawan tribunpalu com asnawi zikri t...
596    banggai raya lebih pebulutangkis dari kabupate...
598    laporan wartawan tribunpalu com asnawi zikri t...
599    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_double_ws, Length: 579, dtype: object

In [ ]:
df.tail()

,target,title,text,lwr,clean_number,clean_punct,clean_double_ws
594,banggai,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...
595,banggai,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
596,banggai,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupate...
598,banggai,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
599,banggai,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...


## Stopword Removal

### Get custom stopwords

In [ ]:
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

In [ ]:
url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

In [ ]:
df_stopword.info()

In [ ]:
df_stopword

In [ ]:
df.head()

In [ ]:
custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

### Removal

In [ ]:
# Prepare stopword remover
#factory = StopWordRemoverFactory()
#stopwords = factory.get_stop_words() + custom_stopword
#print(stopwords)

In [ ]:
# using only sastrawy
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2

In [ ]:
len(stopwords)

946

In [ ]:
dictionary = ArrayDictionary(stopwords)
stopword = StopWordRemover(dictionary)


In [ ]:
stopwords

In [ ]:
def filtering(text):
  stop = stopword.remove(text)
  return stop

In [ ]:
df.head()

In [ ]:
#df.drop(['filtered'], axis=1, inplace=True)

In [ ]:
df['filtered'] = df['clean_double_ws'].apply(filtering)
df['filtered']

0      luwuk suasana pasar murah bersubsidi semarak k...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir penerbitan sttp luwuk...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
594    raya gudang bahan aluminium jalan teuku umar k...
595    wartawan asnawi tribunpalu com banggai membuka...
596    raya pebulutangkis kabupatenkota sulawesi berl...
598    wartawan asnawi tribunpalu com banggai pria ne...
599    wartawan asnawi tribunpalu com banggai gudang ...
Name: filtered, Length: 579, dtype: object

In [ ]:
df.head()

In [ ]:
len(df)

579

In [ ]:
print(df['clean_double_ws'][55])
print(df['filtered'][55])

## Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [ ]:
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()

In [ ]:
def stem(text):
  return stemmer.stem(text)

In [ ]:
df.head()

In [ ]:
df['stemmed'] = df['filtered'].apply(stem)
df['stemmed']

0      luwuk suasana pasar murah subsidi semarak hadi...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir terbit sttp luwuk rak...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
594    raya gudang bahan aluminium jalan teuku umar k...
595    wartawan asnawi tribunpalu com bangga buka aca...
596    raya pebulutangkis kabupatenkota sulawesi laga...
598    wartawan asnawi tribunpalu com bangga pria nek...
599    wartawan asnawi tribunpalu com bangga gudang b...
Name: stemmed, Length: 579, dtype: object

In [ ]:
df.head()

## Export df to csv (to cutting time if we want to stepback process)

In [ ]:
df.to_csv("stemmed_words_sentiment1.csv", index=False)

## Importing df rom csv (cutting time if we want to stepback process)

In [ ]:
# to skip stemming process
stemmed_url= 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/stemmed_words2_tanpa_token_stopwordk2.csv'

df = pd.read_csv(stemmed_url,encoding='utf-8')

df = df.astype(str)

In [ ]:
df.head()


In [ ]:
len(df)

149

In [ ]:
print(df['filtered'][55])
print(df['stemmed'][55])

tim inafis polres banggaiselidiki pencurian desa bubung kerugian rp juta tim indonesianautomatic fingerprints identification system inafissatreskrimpolres banggaitengah intensif penyelidikan terkait pencurian terjadi didesa bubungkecamatan luwuk selatan kejadian diketahui kamis wita dilaporkan nomor polisi lpbxiispktres banggaipolda sulteng kamis desember keterangankasat reskrim polres akp tondytim inafis olah kejadian perkara tkp mengumpulkan bukti petunjuk terkait kejadian korban nama arifin morintoh melaporkan kejadian pada pukul wita menemukan rumahnya telah dirusak pencuri saksi saksi terlibat kejadian seperti indah permata sari morintoh keterangan saat rumahnya di desa bubung menemukan rumah berantakan ditemukannya jendela rumah samping dirusak cara dicungkil akp anjas suyono penjaga rumah menjelaskan pada rabu pukul wita meninggalkan rumah dalam keadaan terkunci ia terkejut indah memberitahu rumah telah sasaran pencurian akibat kejadian hilang unit keyboard merk yamaha kg kemiri

In [ ]:
print(len(df['filtered'][796]))
print(len(df['stemmed'][796]))

1167
990


## Tokenizing

In [ ]:
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [ ]:
df.head()

In [ ]:
df['tokenized'] = df['stemmed'].apply(tokenize)
df['tokenized']

0      [luwuk, suasana, pasar, murah, subsidi, semara...
1      [luwuk, film, horor, piara, mayat, kreativitas...
2      [luwuk, festival, malabot, tumbe, meriahkankab...
3      [kerja, cepat, terkoordinir, terbit, sttp, luw...
4      [luwuk, bawaslu, banggaiperkuat, koordinasi, d...
                             ...                        
594    [raya, gudang, bahan, aluminium, jalan, teuku,...
595    [wartawan, asnawi, tribunpalu, com, bangga, bu...
596    [raya, pebulutangkis, kabupatenkota, sulawesi,...
598    [wartawan, asnawi, tribunpalu, com, bangga, pr...
599    [wartawan, asnawi, tribunpalu, com, bangga, gu...
Name: tokenized, Length: 579, dtype: object

In [ ]:
print(df['filtered'][30])
print(df['tokenized'][30])

dinas sosial banggaisegera salurkan miliarprogram gercep gaskan berdaya dinas sosial banggaimenerima bantuan miliar rupiah program gerak cepat pengentasan kemiskinan berbasis pemberdayaan masyarakat gercep gaskan berdaya bantuan ditujukan keluarga penerima manfaat lima kecamatan harapan warga kecamatan kpm luwuk selatan kpm pagimana kpm bunta kpm nuhon kpm selasa kepala dinas sosial irpan poma kepala bidang fakir miskisronald putjemenyampaikan media dana bantuan sumbangan langsung pemerintah provinsi sulawesi dinas sosial bertanggung memfasilitasi distribusi bantuan ikut pengawasan ketat dana sampai penerima manfaat tepat data tersedia keluarga penerima manfaat terdiri lima kecamatan memenuhi kriteria menerima bantuan dinas sosial berharap bantuan bantuan nyata meringankan beban masyarakat membutuhkan bantuan sekadar alokasi dana merupakan upaya konkret upaya mengentaskan kemiskinan pemberdayaan masyarakat semoga kuncuran dana masyarakat penerima manfaat merasakan dampak positifnya mem

In [ ]:
print(len(df['stemmed'][30]))
print(len(df['tokenized'][30]))

1788
260


##Second Stopwords

In [ ]:
df.head()

In [ ]:
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
df['filtered2'] = df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

# Display the updated DataFrame
print(df.head())

In [ ]:
#df.drop(columns=['tokenized'], inplace=True)

In [ ]:
len(stopwords)

946

In [ ]:
df.head()

In [ ]:
print(df['tokenized'][594])
print(df['filtered2'][594])

['raya', 'gudang', 'bahan', 'aluminium', 'jalan', 'teuku', 'umar', 'kompleks', 'toko', 'lurah', 'kabupaten', 'alami', 'bakar', 'jumat', 'pukul', 'wita', 'plt', 'kapolsek', 'akp', 'steven', 'lewaherillah', 'terang', 'saksi', 'andui', 'muncul', 'titik', 'api', 'bagi', 'samping', 'bangun', 'gudang', 'milik', 'donal', 'dusia', 'warga', 'jalan', 'jendral', 'ahmad', 'yani', 'toko', 'mas', 'jawa', 'baca', 'truk', 'angkut', 'ton', 'raskin', 'perosok', 'selokan', 'pagimana', 'titik', 'api', 'muncul', 'dari', 'tumpu', 'gabus', 'ikan', 'samping', 'bangun', 'gudang', 'milik', 'handies', 'umaiyer', 'baca', 'ihk', 'kota', 'alami', 'inflasi', 'rilis', 'lengkap', 'bpssementara', 'kata', 'steven', 'bahwa', 'itu', 'saksi', 'dengar', 'ria', 'keluar', 'api', 'besar', 'bagi', 'samping', 'gudang', 'langsung', 'hubung', 'madam', 'bakar', 'api', 'cepat', 'padam', 'tidak', 'luas', 'bagi', 'samping', 'yang', 'bakar', 'baca', 'curi', 'receiver', 'cctv', 'wifi', 'pria', 'maahas', 'bekuk', 'polisiia', 'rugi', 'mat

# Text Classification

In [ ]:
# For manual Review
#import random

# Set the number of random samples you want to select
#num_samples = 100  # Adjust this number as needed

# Select random samples from the DataFrame
#random_indices = random.sample(range(len(df)), num_samples)
#random_data = df.iloc[random_indices][["title", "text"]]

# Save random data to a CSV file
#random_data.to_csv("random_data.csv")


In [ ]:
df.info()

## Preprocessing (Text Classification)

In [ ]:
df.head()

In [ ]:
# Spliting database to train_set and test_set
from sklearn.model_selection import train_test_split

X = df['filtered2']
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)

In [ ]:
X_train.head()

95     [lembaga, jamin, mutu, kembang, didik, lpmpp, ...
104    [program, studi, prodi, manajemen, fakultas, e...
66     [juni, inflasi, year, on, year, yoy, kota, cat...
0      [warga, jaring, utara, nama, josua, bangun, in...
141    [anggota, satlantaspolres, banggaimelaksanakan...
Name: filtered2, dtype: object

In [ ]:
print(y_test.unique())


#df['Topic'].nunique()

In [ ]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

# Fit Vectorizer to the data (build a vocab, count the number of words..)
# and transfrom the original text message to ---> Vector

from itertools import chain

# Assuming X_train is a list of lists or a list of tokens
flattened_X_train = list(chain.from_iterable(X_train))
flattened_X_test = list(chain.from_iterable(X_test))

# Assuming X_train is a list of tokens
X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

X_train_counts = count_vect.fit_transform(X_train)


In [ ]:
X_train_counts

<99x3291 sparse matrix of type '<class 'numpy.int64'>'
	with 8942 stored elements in Compressed Sparse Row format>

In [ ]:
X_train.shape

(99,)

In [ ]:
X_train_counts.shape

(99, 3291)

In [ ]:
# TFIDFVectorizer and LinearSVC with pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train, y_train)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
# Checking performance using confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(f"Accuracy {metrics.accuracy_score(y_test, predictions)}")

[[ 4  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0]
 [ 0  0 16  0  1  1  0  0]
 [ 1  0  0  4  0  0  0  0]
 [ 3  0  2  0  5  1  0  0]
 [ 0  0  0  0  0  4  0  0]
 [ 0  0  2  0  0  0  0  0]
 [ 0  2  1  0  0  0  0  0]]
                                          precision    recall  f1-score   support

                           Ekonomi Lokal       0.50      1.00      0.67         4
Infrastruktur, Pembangunan dan Perumahan       0.60      1.00      0.75         3
             Keamanan dan Layanan Publik       0.76      0.89      0.82        18
                        Kesehatan Publik       1.00      0.80      0.89         5
        Lingkungan dan Partisipasi Warga       0.83      0.45      0.59        11
                              Pendidikan       0.67      1.00      0.80         4
                                 Politik       0.00      0.00      0.00         2
                            Transportasi       0.00      0.00      0.00         3

                                accuracy   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Input Data

In [ ]:
# Input
text_url = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/input_text.csv'

# Read CSV file with 'latin-1' encoding
input_df = pd.read_csv(text_url, encoding='utf-8')

# Convert all columns to string
input_df = input_df.astype(str)

In [ ]:
input_df.head()

## Preprocessing

In [ ]:
# Delete Duplicates
input_df.duplicated(subset='title').sum()
input_df = input_df.drop_duplicates(subset='title')

# Delete Blank text (Nan) Rows
input_df = input_df[input_df['text'] != '']

# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

input_df['lwr'] = input_df['text'].apply(clean_lower)
input_df['lwr']


# Remove Numbers
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

input_df['clean_number'] = input_df['lwr'].apply(remove_numbers)
input_df['clean_number']


# Remove Punctuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

input_df['clean_punct'] = input_df['clean_number'].apply(clean_punct)
input_df['clean_punct']


# Delete Whitespaces
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

input_df['clean_double_ws'] = input_df['clean_punct'].apply(normalize_whitespace)
input_df['clean_double_ws']


In [ ]:
len(input_df)

In [ ]:
input_df.head()

In [ ]:
# Adding Stopwords
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2


# 1st Stopwords Removal
def filtering(text):
  stop = stopword.remove(text)
  return stop

input_df['filtered'] = input_df['clean_double_ws'].apply(filtering)
input_df['filtered']


# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def stem(text):
  return stemmer.stem(text)

input_df['stemmed'] = input_df['filtered'].apply(stem)
input_df['stemmed']


# Tokenizing
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

input_df['tokenized'] = input_df['stemmed'].apply(tokenize)
input_df['tokenized']


# 2nd Stopwords Removal
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
input_df['filtered2'] = input_df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

In [ ]:
input_df.head()

In [ ]:
#input_df.drop(columns=['predictions'], inplace=True)

In [ ]:
# Convert tokenized text to string representation
#text_strings = [' '.join(doc) for doc in df['tokenized']]
input_df['text2'] = input_df['tokenized'].apply(lambda x: ' '.join(x))


In [ ]:
input_df.head()

# Predicting Input Data

In [ ]:

# Assuming 'selected_features' is a list of feature names used during model training
X_input = input_df['text2']

# 3. Predict using the model
predictions = text_clf.predict(X_input)

# 4. Add predictions to a new column in the DataFrame
input_df['topic'] = predictions

In [ ]:
input_df.head()



```
# This is formatted as code
```

## Convert input_df to csv

In [ ]:
# Assuming 'df' is your existing DataFrame and 'columns_to_keep' is a list of column names you want to keep
columns_to_keep = ['title', 'text', 'topic']  # Adjust with your desired column names

# Create a new DataFrame with only the selected columns
final_text = input_df[columns_to_keep].copy()

final_text.to_csv('final_text.csv', index=False)

In [ ]:
import pickle
#pickle.dump(model, open('model_name_you_want_to_save_as.sav', 'wb'))
pickle.dump(text_clf, open('text_classifier.sav', 'wb'))